In [2]:
import pandas as pd

In [3]:
# 讀取CB資料
path_cb = r'C:\Users\User\Desktop\CB-pricing\CB_mess.txt'
cb_df = pd.read_csv(path_cb, encoding='big5', sep='\t', encoding_errors='replace')
cb_df


C:\Users\User\AppData\Local\Temp\ipykernel_17692\1185916436.py:3: DtypeWarning: Columns (3,8,15,16,17,21,22,23,24,28,29,41) have mixed types. Specify dtype option on import or set low_memory=False.
  cb_df = pd.read_csv(path_cb, encoding='big5', sep='\t', encoding_errors='replace')


,公司代碼,簡稱,年月,最近一期轉換價格日,最近一期配股基準日,首次發行(Y/N),發行總額(1000),到目前餘額(1000),轉換價格,上市日,...,賣回價%(五),賣回收益率五,賣回日起(六),賣回日迄(六),賣回價%(六),賣回收益率六,賣回日起(七),賣回日迄(七),賣回價%(七),賣回收益率七
0,0AH781P,臺醫光電一(P),201910,20191018,2019/10/18,Y,10000,10000,2.7,,...,-,-,,,-,-,,,-,-
1,0AH781P,臺醫光電一(P),201911,20191018,2019/10/18,N,10000,10000,2.7,,...,-,-,,,-,-,,,-,-
2,0AH781P,臺醫光電一(P),201912,20191018,2019/10/18,N,10000,10000,2.7,,...,-,-,,,-,-,,,-,-
3,0AH781P,臺醫光電一(P),202001,20191018,2019/10/18,N,10000,10000,2.7,,...,-,-,,,-,-,,,-,-
4,0AH781P,臺醫光電一(P),202002,20191018,2019/10/18,N,10000,0,2.7,,...,-,-,,,-,-,,,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103715,Z55911P,富盛國際產業一(P),202411,20240715,2024/07/15,N,2000000,2000000,11.0000,,...,-,-,,,-,-,,,-,-
103716,Z55921P,以諾驛站一(P),202408,20240810,2024/08/10,Y,6000,6000,10.0000,,...,-,-,,,-,-,,,-,-
103717,Z55921P,以諾驛站一(P),202409,20240810,2024/08/10,N,6000,6000,10.0000,,...,-,-,,,-,-,,,-,-
103718,Z55921P,以諾驛站一(P),202410,20240810,2024/08/10,N,6000,6000,10.0000,,...,-,-,,,-,-,,,-,-


In [4]:
cb_df = cb_df[cb_df['公司代碼'].str.contains('6753')]
cb_df

,公司代碼,簡稱,年月,最近一期轉換價格日,最近一期配股基準日,首次發行(Y/N),發行總額(1000),到目前餘額(1000),轉換價格,上市日,...,賣回價%(五),賣回收益率五,賣回日起(六),賣回日迄(六),賣回價%(六),賣回收益率六,賣回日起(七),賣回日迄(七),賣回價%(七),賣回收益率七
91480,67531,龍德造船一,202406,20240625,2024/06/25,Y,1000000,1000000,125.0,20240625,...,-,-,,,-,-,,,-,-
91481,67531,龍德造船一,202407,20240625,2024/06/25,N,1000000,1000000,125.0,20240625,...,-,-,,,-,-,,,-,-
91482,67531,龍德造船一,202408,20240625,2024/06/25,N,1000000,1000000,125.0,20240625,...,-,-,,,-,-,,,-,-
91483,67531,龍德造船一,202409,20240901,2024/09/01,N,1000000,1000000,117.9,20240625,...,-,-,,,-,-,,,-,-
91484,67531,龍德造船一,202410,20240901,2024/09/01,N,1000000,1000000,117.9,20240625,...,-,-,,,-,-,,,-,-
91485,67531,龍德造船一,202411,20240901,2024/09/01,N,1000000,1000000,117.9,20240625,...,-,-,,,-,-,,,-,-


In [5]:
cb_df.to_csv('cb_6751_test.csv')